In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
torch.manual_seed(1)    # reproducible

In [27]:
# Hyper Parameters
EPOCH = 1               # train the training data n times, to save time, we just train 1 epoch
BATCH_SIZE = 1
TIME_STEP = 1          # rnn time step / image height
INPUT_SIZE = 8         # rnn input size / image width
LR = 0.01               # learning rate

In [28]:
train_x=np.array([[0.129779 ,0.352941 ,0.245902 ,0.527273 ,0.666667 ,0.002290 ,0.000000 ,0.0],  
        [0.148893 ,0.367647 ,0.245902 ,0.527273 ,0.666667 ,0.003811 ,0.000000 ,0.0],
        [0.159960 ,0.426471 ,0.229508 ,0.545454 ,0.666667 ,0.005332 ,0.000000 ,0.0],
        [0.182093 ,0.485294 ,0.229508 ,0.563637 ,0.666667 ,0.008391 ,0.037037 ,0.0],
        [0.138833 ,0.485294 ,0.229508 ,0.563637 ,0.666667 ,0.009912 ,0.074074 ,0.0]],dtype=np.float32)
train_y=np.array([0.148893 ,0.159960 ,0.182093 ,0.138833 ,0.109658],dtype=np.float32)

In [29]:
train_y

array([0.148893, 0.15996 , 0.182093, 0.138833, 0.109658], dtype=float32)

In [30]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=INPUT_SIZE,
            hidden_size=64,         # rnn hidden unit
            num_layers=1,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.out = nn.Linear(64, 1)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return out

In [31]:
rnn = RNN()
print(rnn)

RNN(
  (rnn): LSTM(8, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=1, bias=True)
)


In [32]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

In [33]:
train_x=torch.from_numpy(train_x)
train_y=torch.from_numpy(train_y)

In [37]:
for step in range(100):        # gives batch data
        b_x = Variable(train_x[step])              # reshape x to (batch, time_step, input_size)
        b_y = Variable(train_y[step])                             # batch y

        output = rnn(b_x)                               # rnn output
        loss = loss_func(output, b_y)                   # cross entropy loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients
        
        
        if step % 10 == 0:
            print(loss)

RuntimeError: input must have 3 dimensions, got 1